# FLIP (04): Learning Theory (I)
**(Module 03: Operation Research)**

---
- Materials in this module include resources collected from various open-source online repositories.
- You are free to use, but NOT allowed to change or distribute this package.

Prepared by and for 
**Student Members** |
2006-2018 [TULIP Lab](http://www.tulip.org.au), Australia

---

# Case Study - Landhills Winery

This notebook demonstrates the use of linear programming to analyze the case study

>Owen Hall and Kenneth Ko. "Landhills Winery: Developing an Optimal Blending Plan," 3 pages, HBS Case W14167-PDF-ENG, published May 21, 2014.

The notebook uses MathProg/GLPK to represent the model and calculate solutions.

### Initializations

In [ ]:
%matplotlib inline
from pylab import *

from IPython.core.display import HTML
HTML(open("styles/custom.css", "r").read())

In [ ]:
%%script glpsol -m /dev/stdin -o /dev/stdout -y display.txt --out output

set GRAPES;
set WINES;
    
param gAcid{GRAPES};
param gSugr{GRAPES};
param gAlch{GRAPES};
param gQnty{GRAPES};
param gCost{GRAPES}; 

param wAcid{WINES};
param wSugr{WINES};
param wAlch{WINES};
param wPric{WINES};
   
var x{WINES,GRAPES}, integer >= 0;
var wProd{WINES} >= 0;
var gCons{GRAPES} >= 0;

maximize Profit : 
    sum{w in WINES} wPric[w]*wProd[w] - sum{g in GRAPES} gCost[g]*gCons[g];

# balances
subject to grapes {g in GRAPES} : 
    sum{w in WINES} x[w,g] == gCons[g];
subject to wines {w in WINES} : 
    sum{g in GRAPES} x[w,g] == wProd[w];

# bound on grapes
subject to capacity {g in GRAPES} : 
    gCons[g] <= gQnty[g];

# acidity
subject to acidity {w in WINES} : 
    sum{g in GRAPES} x[w,g]*gAcid[g] <= wProd[w]*wAcid[w];

# sugar
subject to sugar {w in WINES} : 
    sum{g in GRAPES} x[w,g]*gSugr[g] <= wProd[w]*wSugr[w];

# Varietal content
subject to v1 {w in {'wSB11','wSL10','wSL11','wCabS'}} : 
    sum {g in {'gSB11','gSL10','gSL11'}} x[w,g] >= 0.75*wProd[w];

subject to v2 {w in {'wMerl'}} : 
    sum {g in {'gMerl'}} x[w,g] >= 0.75*wProd[w];

# alchohol content
subject to a1 {w in WINES} : 
    sum{g in GRAPES} x[w,g]*gAlch[g] >= 10.0*wProd[w];
subject to a2 {w in WINES} : 
    sum{g in GRAPES} x[w,g]*gAlch[g] <= 15.0*wProd[w];

# vintage
subject to vn1 : 
    x['wSB11','gSB11'] >= 0.95*wProd['wSB11'];
subject to vn2 : 
    x['wSL10','gSL10'] >= 0.95*wProd['wSL10'];
subject to vn3 : 
    x['wSL11','gSL11'] >= 0.95*wProd['wSL11'];

solve;

printf "Profit = %9.2f\n\n", Profit;
for {w in WINES} {
    printf "%6s  ", w;
    printf "%8.1f  ", wProd[w];
    for {g in GRAPES} printf "%8.1f   ", x[w,g];
    printf "\n";
}

data;
    
param : GRAPES : gAcid, gSugr, gAlch,  gQnty, gCost :=
        gSB11     0.35   0.12   13.5   50000   2.35
        gSL10     0.75   0.25   15.3   60000   2.60
        gSL11     0.55   0.30   11.5   30000   2.10
        gMerl     0.25   0.08   15.7  200000   1.55;
    
param : WINES  : wAcid, wSugr, wPric :=
        wSB11      0.7    0.2   9.00
        wSL10      0.7    0.2   9.00
        wSL11      0.7    0.2   9.00
        wCabS      0.7    0.3   5.50
        wMerl      0.3    1.0   2.95;
end;


In [ ]:
print(open('display.txt').read())

In [ ]:
print output

In [ ]:
%%script glpsol -m /dev/stdin -o /dev/stdout -y display.txt --out output

set VARIETALS := {'SB2011','SLO2010','SLO2011'};
set NONVARIETALS := {'Cab','Merlot'};
set CABERNETs := {'SB2011','SLO2010','SLO2011','Cab'};


param price{WINES};

param acidity{GRAPES};
param sugar{GRAPES};
param alcohol{GRAPES};
param quantity{GRAPES};
param cost{GRAPES}; 
    
var x{WINES,GRAPES} >= 0;
var wine{WINES} >= 0;
var grape{GRAPES} >= 0;

maximize PROFIT: sum{w in WINES} wine[w]*price[w] - sum{g in GRAPES} grape[g]*cost[g];

subject to g_cons{g in GRAPES}: sum{w in WINES} x[w,g] = grape[g];
subject to w_prod{w in WINES}: sum{g in GRAPES} x[w,g] = wine[w];
    
subject to acid{w in WINES}

subject to qty{g in GRAPES}: sum{w in WINES} x[w,g] <= quantity[g];

solve;

data;

param : WINES : price :=
    V1   9.00
    V2   9.00
    V3   9.00
    NV   5.50
    M    2.95;

param : GRAPES : acidity, sugar, alcohol, quantity, cost :=
    G1   0.35   0.12   13.5    50000   2.35
    G2   0.75   0.25   15.3    60000   2.60
    G3   0.55   0.30   11.5    30000   2.10
    G4   0.25   0.08   15.7   200000   1.55;
 
end;


Looking at the model output

In [ ]:
print output